In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', 1000)     

In [2]:
df = pd.read_csv('data/01.csv')
df

,id_pool,id_courier,courier_type,id_job,pool_date_start,sort_order,id_general,distance,status_str,date_status,id_general.1,total_order_weight,total_orders,first_date_order,last_date_order,date_supply,date_supply_untill,date_collected,date_delivery_start,date_delivered,id_courier.1,id_service,id_source,latitude_order,longitude_order,N,shirota_tt,dolgota_tt,gen_distance
0,34769836,94477626.0,Авто,258985879,2024-01-29 18:23:16.660,3.0,252371684.0,"1,871",job_posted,2024-01-29 17:52:08.457,252371684.0,6.478,1.0,2024-01-29 17:42:59.400,2024-01-29 17:42:59.400,2024-01-29 17:42:00.000,2024-01-29 18:42:00.000,2024-01-29 17:49:34.387,2024-01-29 17:52:30.490,2024-01-29 18:26:18.703,9.447763e+07,32.0,2.0,55.716801,37.891499,5981.0,55.724824,37.910756,"1,995"
1,34769836,94477626.0,Авто,258984196,2024-01-29 17:49:37.277,1.0,252370826.0,"1,796",job_posted,2024-01-29 17:47:45.763,252370826.0,10.609,1.0,2024-01-29 17:40:58.250,2024-01-29 17:40:58.250,2024-01-29 17:40:00.000,2024-01-29 18:40:00.000,2024-01-29 17:46:30.120,2024-01-29 17:49:18.827,2024-01-29 18:18:03.330,9.447763e+07,32.0,3.0,55.714930,37.895327,5981.0,55.724824,37.910756,"1,79"
2,33310463,94477626.0,Авто,249449879,2024-01-05 09:39:55.817,1.0,243436941.0,"1,751",job_posted,2024-01-05 09:30:45.020,243436941.0,1.443,1.0,2024-01-04 20:59:24.537,2024-01-04 20:59:24.537,2024-01-05 10:00:00.000,2024-01-05 11:00:00.000,2024-01-05 04:59:53.787,2024-01-05 09:39:16.197,2024-01-05 09:57:08.760,9.447763e+07,32.0,3.0,55.714342,37.897004,5981.0,55.724824,37.910756,"1,663"
3,33310463,94477626.0,Авто,249450220,2024-01-05 10:21:46.957,7.0,243413908.0,"0,55",job_posted,2024-01-05 09:30:53.383,243413908.0,11.206,1.0,2024-01-04 19:25:41.997,2024-01-04 19:25:41.997,2024-01-05 10:00:00.000,2024-01-05 11:00:00.000,2024-01-05 05:01:31.583,2024-01-05 09:40:01.587,2024-01-05 10:22:31.890,9.447763e+07,32.0,2.0,55.709231,37.886447,5981.0,55.724824,37.910756,"2,82"
4,33319925,94477626.0,Авто,249512972,2024-01-05 12:48:11.027,8.0,243549906.0,"1,475",job_posted,2024-01-05 11:42:25.683,243549906.0,1.235,1.0,2024-01-05 11:31:19.067,2024-01-05 11:31:19.067,2024-01-05 11:31:00.000,2024-01-05 12:31:00.000,2024-01-05 11:37:07.093,2024-01-05 11:58:34.317,2024-01-05 12:50:17.340,9.447763e+07,32.0,2.0,55.721001,37.891998,5981.0,55.724824,37.910756,"1,51"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307162,34787250,NaN,Тип курьера неизвестен,258258634,NaN,1.0,251618198.0,"4,082",job_posted,2024-01-27 17:11:29.153,251618198.0,1.250,1.0,2024-01-27 16:06:16.840,2024-01-27 16:06:16.840,2024-01-27 16:06:00.000,2024-01-27 18:06:00.000,2024-01-27 16:08:03.003,2024-01-27 17:11:34.647,2024-01-28 09:06:09.390,1.699945e+09,1.0,3.0,55.799322,37.697728,5668.0,55.781688,37.675279,"4,12"
4307163,34787252,NaN,Тип курьера неизвестен,258733083,NaN,1.0,252107083.0,"8,99",job_reposted,2024-01-29 00:03:55.250,252107083.0,2.429,1.0,2024-01-28 22:44:34.777,2024-01-28 22:44:34.777,2024-01-29 06:00:00.000,2024-01-29 08:00:00.000,2024-01-28 22:51:52.333,2024-01-29 06:01:04.637,2024-01-29 06:45:42.340,1.699922e+09,1.0,10.0,55.697688,37.623502,4799.0,55.680996,37.692869,"7,79"
4307164,34727517,NaN,Тип курьера неизвестен,258733083,NaN,1.0,252107083.0,"8,99",job_reposted,2024-01-29 00:03:55.250,252107083.0,2.429,1.0,2024-01-28 22:44:34.777,2024-01-28 22:44:34.777,2024-01-29 06:00:00.000,2024-01-29 08:00:00.000,2024-01-28 22:51:52.333,2024-01-29 06:01:04.637,2024-01-29 06:45:42.340,1.699922e+09,1.0,10.0,55.697688,37.623502,4799.0,55.680996,37.692869,"7,79"
4307165,34667086,NaN,Тип курьера неизвестен,258258634,NaN,1.0,251618198.0,"4,082",job_posted,2024-01-27 17:11:29.153,251618198.0,1.250,1.0,2024-01-27 16:06:16.840,2024-01-27 16:06:16.840,2024-01-27 16:06:00.000,2024-01-27 18:06:00.000,2024-01-27 16:08:03.003,2024-01-27 17:11:34.647,2024-01-28 09:06:09.390,1.699945e+09,1.0,3.0,55.799322,37.697728,5668.0,55.781688,37.675279,"4,12"


#### проверить

Часто ли бывает, что курьер работает на нескольких ТТ и берет с 1 ТТ - оч мало заказов ?

### Обработка по курьерам

В конце нужно будет иметь таблицу со следующими данными:
ид курьера, тт, дата, время, статус, колво заказов на весь день в тот ТТ, колво пулов на весь день в тот ТТ, сколько часов работал на тот день, 

Статистики, которые нужно будет отдельно считать и потом джоинить с таблицей выше:
- Среднее кол-во заказов в час 
- Среднее кол-во пулов в час
- Ср кол-во часов в смене в будни 
- Ср кол-во часов в смене в выходные 
- Кол-во смен в день ( утром + вечер и тд)
- Ср вес заказа
- Ср. время доставки 1 пула
- Ср. время доставки 1 заказа
- Ср. время между взятие следующего пула или заказа и delivered
- Ср. скорость доставки 
- ср. расстояние до заказа (gen_distance)


До того чтобы начинать надо исправить один момент:
Есть два столбцы с ид курьера (id_courier из таблицы с пулями, id_courier.1 из Reports_data). Иногда бывает что ид есть только в id_courier.1 и наоборот иногда только в id_courier, поэтому нужно дополнить один из них со значенями которых нет. Если данные о курьере есть в id_courier.1, а не в id_courier означает, что нужно еще получить данные о курьере (courier_type) из исторических данных об этом курьере, так как из-за того что там null в id_courier, тип курьера будет неизвестен.

После этого нужно дропнуть строки где нигде нет инфо о курьере

In [ ]:
# код


Также нужно удалить строки где date_status до date_order - это аномалия, которая означает, что заказ появился в приложение курьера до того когда этот заказ заказал клиент

In [ ]:
# код

Статистики:
Добавим дату в формате datetime (date_group) для каждой строки на основе date_delivery_start (когда курьер выбрал заказ), если дата 01.02.2024 13:24:22 значит дата и время будет 01.02.2024 13:15, т.е 
hh:00-hh:15 - пишется как hh:00
hh:15-hh:30 - пишется как hh:15
hh:30-hh:45 - пишется как hh:30
hh:45-hh:00 - пишется как hh:45



По группе id_courier, tt, date_group считать колво выбранных заказов курьерам (date_delivery_start)


Для того чтобы определить колво пулов нужно будет добавить тоже дату начала и конца пула.
- чтобы понять время начала пула можно смотреть на date_delivered первого заказа в пуле и на основе gen_distance вычесть несколько минут чтобы учитывать когда он из ТТ уехал (date_start_pool)
- чтобы понять время конца пула смотреть на date_delivered последнего заказа и добавить несколько минут также на основе gen_distance (date_finish_pool)




Считать время возврата курьера в ТТ.
Для этого можно использовать date_delivered последнего заказа, gen_distance, или самим попробовать считать distance между координатами чтобы убедиться что gen_distance реально показывает дистанцию между заказом и тт.

Зачем считать? Мы можем потом найти среднее время возврата и на основе этого понять когда он заканчивал смену 

На основе время начала и конца добавить столбец статуса курьера:
- в пути (если date_group между date_start_pool)
- в тт (если есть date_delivery_start какого-то заказа и date_finish_pool до date_delivery_start)
- дома 🐣 (если за больше чем среднее время возврата в ТТ он не выбрал заказы)

Также добавить для каждого date_group получил ли пул курьера (has_pool)? 1 - если да, 0 - если нет, чтобы потом считать колво пулов по группе id_courier, tt, date_group (просто sum has_pool)

### Обработка по ТТ